In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, random_split

# Import Numpy
import numpy as np
# For timer
import time
# Import OS
import os

# ==== LOCAL LIBS ====
import ecg_tools_lite as et
import evaluator as eva
import trainer as tr
import models as mod

# Summary-like from Tensorflow
from torchsummary import summary

In [2]:
# What are you trying to do?
create_new_model = False
create_new_res = False

# If create_new_model is True, epochs must have a value
epochs = 10

# Model used:
model_net = mod.ae_sampler

# If create_new_model is False, load_model_name must have a value
load_model_name = 'model_2021-3-14_150.pt'
ecg_saved_res = 'res_pt_full_2021-3-14_202.npy'

# Normalization option (either option 0 or 1)
norm_option = 2

In [3]:
# # combine the ultimate noise thing
# ecg_n6 = np.load('all_none_fs1024_en6.npy')
# ecg_00 = np.load('all_none_fs1024_e00.npy')
# ecg_06 = np.load('all_none_fs1024_e06.npy')
# ecg_12 = np.load('all_none_fs1024_e12.npy')
# ecg_18 = np.load('all_none_fs1024_e18.npy')
# ecg_24 = np.load('all_none_fs1024_e24.npy')

# ecg_noisy = np.concatenate((ecg_n6, ecg_00, ecg_06, ecg_12, ecg_18, ecg_24))
# ecg_clean = np.concatenate((ecg_24, ecg_24, ecg_24, ecg_24, ecg_24, ecg_24))
# np.save('all_clean_em', ecg_clean)

In [4]:
ecg_noisy = torch.from_numpy(np.load('all_noise_em.npy')).cuda().double()
ecg_clean = torch.from_numpy(np.load('all_clean_em.npy')).cuda().double()

# Acquire original shape
x = ecg_noisy.shape[0]
y = ecg_noisy.shape[2]
z = ecg_noisy.shape[1]

# Reshape
ecg_noisy = torch.reshape( ecg_noisy, (x, y, z))
ecg_clean = torch.reshape( ecg_clean, (x, y, z))

print(f'New shape: {ecg_noisy.shape}')

# == Normalization technique (Norm from -1 to 1)
ecg_noisy = et.norm_sig( ecg_noisy )
ecg_clean = et.norm_sig( ecg_clean )

New shape: torch.Size([33264, 1, 1024])


In [5]:
if create_new_model:
    model = et.train_model( model=model_net,
                            epochs=epochs,
                            ecg_noisy=ecg_noisy,
                            ecg_clean=ecg_clean)
else:
    model = et.load_model( model_name=load_model_name,
                            model=model_net)

train_size[26611] + valid_size[6653]= 33264
same size
Epoch 1 of 10 || time: 434.90 || loss = 0.1198025223014135
Epoch 2 of 10 || time: 464.34 || loss = 0.10028949718295589
Epoch 3 of 10 || time: 567.08 || loss = 0.09803775082469599
Epoch 4 of 10 || time: 511.07 || loss = 0.09658521927716855
Epoch 5 of 10 || time: 557.52 || loss = 0.0952991406029814
Epoch 6 of 10 || time: 553.42 || loss = 0.09339121775303696
Epoch 7 of 10 || time: 526.62 || loss = 0.09263021453216269
Epoch 8 of 10 || time: 570.54 || loss = 0.09205506802113116
Epoch 9 of 10 || time: 557.57 || loss = 0.0915188709365473
Epoch 10 of 10 || time: 572.10 || loss = 0.09113226381556154
Elapsed time: 5315.19, (in mins: 88:35)
Validation dataset has not been used: Available validex set size = 6653
Saved model_2021-3-14_452.pt


In [6]:
# summary( model )

In [7]:
if create_new_model == True or create_new_res == True:
    ecg_res = et.concat_pt_full(model, ecg_noisy)
    et.get_local_time()
else:
    ecg_res = np.load( ecg_saved_res )

ecg_res = ecg_res.flatten()

Result size: (2000, 1, 1024)
Result size: (2000, 1, 1024)
Result size: (2000, 1, 1024)
Result size: (2000, 1, 1024)
Result size: (2000, 1, 1024)
Result size: (2000, 1, 1024)
Result size: (2000, 1, 1024)
Result size: (2000, 1, 1024)
Result size: (2000, 1, 1024)
Result size: (2000, 1, 1024)
Result size: (2000, 1, 1024)
Result size: (2000, 1, 1024)
Result size: (2000, 1, 1024)
Result size: (2000, 1, 1024)
Result size: (2000, 1, 1024)
Result size: (2000, 1, 1024)
Result size: (2000, 1, 1024)
Result size: (1264, 1, 1024)
Complete shape is: (35264, 1, 1024)
Filename: res_pt_full_2021-3-14_452


In [8]:
# Move to tensor for CUDA to compute

ecg_res = torch.from_numpy( ecg_res )
ecg_res = ecg_res.cuda()

In [9]:
ecg_res

tensor([-0.3436, -0.3688, -0.4000,  ..., -0.5196, -0.4837, -0.4412],
       device='cuda:0', dtype=torch.float64)

In [10]:
with torch.cuda.device(0):
    evals = eva.get_eval_metrics(ecg_clean=ecg_clean.flatten(), ecg_noisy=ecg_noisy.flatten(), ecg_denoised=ecg_res)